In [1]:
from torchtext import data
import pandas as pd
import torch

In [2]:
TEXT = data.Field(sequential=True, tokenize='spacy', pad_first=True, lower=True)
LABEL = data.LabelField(dtype = torch.float, use_vocab=False, sequential=False)

In [3]:
tr = pd.read_csv('train.csv')
print(tr.columns)

Index(['idx', 'DECISION', 'DISCUSSION', 'FINDINGS OF FACT', 'ISSUE', 'JUNK',
       'target'],
      dtype='object')


Below, we associate each column in the csvs with either `None`, `TEXT`, or `LABEL`, depending on which it is. `None` is for columns we don't actually use.

In [4]:
fields = [('idx', None),
          ('DECISION', None),
          ('DISCUSSION', TEXT),
          ('FINDINGS OF FACT', None),
          ('ISSUE', TEXT),
          ('JUNK', None),
          ('target', LABEL)]

train, val = data.TabularDataset.splits(
        format='csv', skip_header=True, path='./',
        train='train.csv', validation='test.csv',
        fields=fields)

Here we load some pretrained vectors from Glove. They're 100 bits long, which we use later in `EMBEDDING_DIM`.

In [5]:
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = 'glove.6B.100d',
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train, val), batch_size=BATCH_SIZE, device=device,
    sort_key=lambda x: len(x.DISCUSSION))

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)
        return self.fc(pooled)

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100 # GLOVE dimension
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [9]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {} trainable parameters.'.format(count_params(model)))

The model has 1000301 trainable parameters.


In [10]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings) # This is preloading these weights into the model?

tensor([[ 0.7052, -0.2839,  2.0560,  ...,  0.6116, -0.3029, -0.2810],
        [-0.5656,  0.1957, -0.4377,  ..., -0.9350,  0.6534,  0.1831],
        [-0.0796,  1.0425,  1.1980,  ..., -0.6556,  0.7965,  0.3770],
        ...,
        [ 0.3633,  0.2066,  0.1472,  ..., -0.5469, -0.6438,  0.2610],
        [ 0.2173, -0.1686, -0.1560,  ...,  0.0951, -0.1963, -0.3810],
        [ 1.8296, -1.0813,  0.2261,  ..., -0.5526, -1.4516,  0.6869]])

In [11]:
# Here we zero out the weights of UNK and PAD, so they can't affect our model
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [13]:
import time

def binary_accuracy(pr, y):
    rounded_pr = torch.round(torch.sigmoid(pr))
    correct = (rounded_pr == y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() # what is this?
    
    for batch in iterator:
        # the batch.X attribute names are from the column names above
        optimizer.zero_grad()
        pr = model(batch.DISCUSSION).squeeze(1) # WTH is squeeze?
        loss = criterion(pr, batch.target)
        acc = binary_accuracy(pr, batch.target)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() # what is this?
    
    with torch.no_grad(): # and this?
        for batch in iterator:
            # the batch.X attribute names are from the column names above
            pr = model(batch.DISCUSSION).squeeze(1)
            loss = criterion(pr, batch.target)
            acc = binary_accuracy(pr, batch.target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    mins = int(elapsed_time / 60)
    secs = int(elapsed_time % 60)
    return mins, secs

In [14]:
N_EPOCHS = 7
best_valid_loss = float('inf')

for e in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    mins, secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tutorial_model.pt')
        
    print('Epoch: {} | Epoch Time: {}m {}s'.format(e, mins, secs))
    print('\tTrain Loss: {:.3f} | Train Acc: {:.3f}'.format(train_loss, train_acc))
    print('\tValid Loss: {:.3f} | Valid Acc: {:.3f}'.format(valid_loss, valid_acc))

Epoch: 0 | Epoch Time: 0m 1s
	Train Loss: 0.683 | Train Acc: 0.660
	Valid Loss: 0.640 | Valid Acc: 0.730
Epoch: 1 | Epoch Time: 0m 1s
	Train Loss: 0.667 | Train Acc: 0.666
	Valid Loss: 0.605 | Valid Acc: 0.730
Epoch: 2 | Epoch Time: 0m 1s
	Train Loss: 0.661 | Train Acc: 0.666
	Valid Loss: 0.586 | Valid Acc: 0.730
Epoch: 3 | Epoch Time: 0m 1s
	Train Loss: 0.654 | Train Acc: 0.664
	Valid Loss: 0.577 | Valid Acc: 0.730
Epoch: 4 | Epoch Time: 0m 1s
	Train Loss: 0.652 | Train Acc: 0.664
	Valid Loss: 0.573 | Valid Acc: 0.730
Epoch: 5 | Epoch Time: 0m 1s
	Train Loss: 0.651 | Train Acc: 0.665
	Valid Loss: 0.572 | Valid Acc: 0.730
Epoch: 6 | Epoch Time: 0m 1s
	Train Loss: 0.648 | Train Acc: 0.663
	Valid Loss: 0.570 | Valid Acc: 0.730
